In [1]:
%%capture
!pip install tensorboardX simpletransformers sklearn

In [2]:
!cat /usr/local/cuda/version.txt
!pip install pydash
!pip install torch==1.6.0
#conda install pytorch==1.1.0 torchvision==0.3.0 cudatoolkit=10.0 -c pytorch
#https://pytorch.org/get-started/previous-versions/

CUDA Version 10.0.130


In [3]:
!rm -rf outputs

In [4]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
from tqdm import tqdm
import pandas as pd

df = pd.read_csv("downloads/47k_train.csv")
len(df)

43558

In [5]:
blacklist=["abstract", "discussion", "background", "summary", "rationale", "introduction", "purpose", "objective" ]
blacklist += [t + ":" for t in blacklist]
def remove_special_words(text, blacklist=blacklist ):
    words = text.split(" ")
    try:
        first=words[0]
    except:
        return text
    
    if any(first.strip().lower() in string for string in blacklist):
        return " ".join(words[1:]).strip()
    return text

remove_special_words("ABSTRACT   The northern corn rootworm")

'The northern corn rootworm'

In [6]:
df["text"] = df["text"].map(remove_special_words)
df= df[["text", "labels"]]

In [7]:
df =df.dropna()
df = df.rename(columns={"labels": "label"})

In [8]:
df= df[["text", "label"]]
df.label = df.label.map(int)

In [9]:
df

,text,label
0,The difficulty with this task lies in the fact...,1
1,The problem with rich annotations is that they...,1
2,"As a consequence , when adapting existing meth...",1
3,The second problem of traditional word alignme...,1
4,The main drawback of these systems is that the...,1
...,...,...
43553,Mobbing is actually a social problem.,0
43554,Students starting off on a tertiary education ...,1
43555,Many individuals with vision impairment experi...,1
43556,Considering China is facing a precipitous decl...,0


In [11]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df2 = df.sample(frac=1)
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. 
#The first column is the text with type str, and the second column is the label with type int.
size = len(df2)
train_cutoff = round(size*0.8)
train_df= df2[:train_cutoff]
eval_df = df2[train_cutoff:]

model_args = ClassificationArgs(save_steps=-1, save_model_every_epoch=False, sliding_window=True)

model = ClassificationModel('distilbert', 'distilbert-base-uncased',args=model_args, use_cuda=True) # You can set class weights by using the optional weight argument

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [13]:
# Train the model
model.train_model(train_df, overwrite_output_dir=True)
# Evaluate the model, u
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/34846 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 34863 features created from 34846 samples.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/4358 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/8712 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 8712 features created from 8712 samples.


Running Evaluation:   0%|          | 0/1090 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.9566285573353129, 'tp': 4425, 'tn': 4098, 'fp': 122, 'fn': 67, 'acc': 0.9783057851239669, 'eval_loss': 0.07401228916589406}


In [14]:
result

{'mcc': 0.9566285573353129,
 'tp': 4425,
 'tn': 4098,
 'fp': 122,
 'fn': 67,
 'acc': 0.9783057851239669,
 'eval_loss': 0.07401228916589406}

In [ ]:
dfx = pd.read_csv("downloads/s2c_5M_bioadjacent.csv",  lineterminator='\n', error_bad_lines=False)

dfx = dfx.dropna(subset=["text"])
dfx=dfx[dfx["text"].str.len() > 15]

len(dfx)

dfx.head()

dfx = dfx[dfx.fields.isin(["Medicine", 'Environmental Science', 'Sociology', 'Psychology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]

dfx["text"] = dfx["text"].map(remove_special_words)

len(dfx)

In [ ]:
dfx["text"] = dfx["text"].astype(str)

In [ ]:
len(dfx)

In [25]:
def stepwise(step, df, f):
    l = []
    for idx in range(step, len(df)+step-1, step):
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx]
        batch = model.predict(list(sl))
        l = l + list(batch[0])
        with open('predicts_backup.txt', 'w') as f:
            for item in l:
                f.write("%s\n" % item)
    return l

In [ ]:
predicts = stepwise(20000, dfx["text"], model.predict)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


0 20000


  0%|          | 0/20000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 20000 features created from 20000 samples.


  0%|          | 0/2532 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


2960000 2980000


  0%|          | 0/20000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (560 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (630 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (551 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 20000 features created from 20000 samples.


  0%|          | 0/2532 [00:00<?, ?it/s]

In [ ]:
with open('downloads/predicts_backup.txt', 'r') as f:
            for item in l:
                preds = f.read().split()#f.write("%s\n" % item)
                

In [ ]:
len(preds)

In [ ]:
dfx = dfx[dfx["predicts"]==1]
dfx = dfx[dfx["text"] != "nan"]
dfx = dfx[dfx["text"].str.len() > 14]
len(dfx)

In [ ]:
dfx.to_csv("downloads/pos_preds_15052022.csv", index = False, header=True)

In [181]:
dt = pd.read_csv("core_pos.csv")
#d[d["first_sent"].str.contains(re.escape("|".join(d["authors"][2:-2].replace("'", "").split(","))).strip().replace('\\', ''))]
def formatStr (x):
    x = eval(x)
    x = [re.escape(v) for v in x]
    return "|".join(x)

regexs = d.authors.apply(formatStr)
matches = pd.Series()
for i, r in tqdm(d.iterrows()):
    matches.loc[i] = len(re.findall(regexs.loc[i], d.first_sent.loc[i])) > 0
dc["authors"] = dc["authors"].map(eval)
dc = dc.drop(columns=["fullText"])
dc.to_csv("core_pos_parsed.csv", index = False, header=True)
tt.to_csv("core_pos_min.csv", index = False, header=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
44962it [00:59, 760.29it/s] 


distilbert
'mcc': 0.7621432780395265, 'tp': 167, 'tn': 480, 'fp': 39, 'fn': 30, 'acc': 0.9036312849162011, 'eval_loss': 0.24669100075132316}

